In [1]:
import pandas as pd
import numpy as np
import operator
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import GradientBoostingRegressor as gbr
from datetime import datetime,timedelta

In [5]:
df0 = pd.read_csv('data_CA_1.csv')
df1 = pd.read_csv('data_CA_2.csv')
df2 = pd.read_csv('data_CA_3.csv')
df3 = pd.read_csv('data_CA_4.csv')
df4 = pd.read_csv('data_TX_1.csv')
df5 = pd.read_csv('data_TX_2.csv')
df6 = pd.read_csv('data_TX_3.csv')
df7 = pd.read_csv('data_WI_1.csv')
df8 = pd.read_csv('data_WI_2.csv')
df9 = pd.read_csv('data_WI_3.csv')

In [3]:
def preprocess(df):
    df.drop(columns = ['Unnamed: 0','store_id','id','weekday','sell_price','event_name_1','event_name_2','event_type_2'],inplace=True)
    
    
    # get the day of month
    date = pd.to_datetime(df['date'],format= '%Y-%m-%d')
    mday = []
    for x in date:
        mday.append(x.strftime("%d"))
    df['mday'] = mday
    df.mday = df.mday.astype(int)
    # get the day number
    df['day'] = df.d.apply(lambda x: x[2:])
    df.day = df.day.astype(int)
    # get the week number
    df['week'] = df.wm_yr_wk.apply(lambda x: x%100)
    
    # dummy to denote national events
    df['national'] = np.where(df.event_type_1=='National',1,0)
    
    df['snap_CA'] = df.snap.apply(lambda x: x[0])
    df['snap_TX'] = df.snap.apply(lambda x: x[2])
    df['snap_WI'] = df.snap.apply(lambda x: x[4])
    
    df.drop(columns = ['d','wm_yr_wk','event_type_1','date','snap'],inplace=True)
    
    return df

In [6]:
df = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
dfs = []
for i in range(10):
    dfs.append(preprocess(df[i]))

In [7]:
def clean(df):

    df_y = df.groupby('day').sum().demand.reset_index(drop = True)
    df_x = df.drop(columns = ['demand']).drop_duplicates().reset_index(drop = True)
    
    df_x['demand_28'] = df_y.shift(28)
    
    df_x = df_x.dropna().reset_index(drop=True)
    df_y = df_y[28:].reset_index(drop=True)

    return df_x,df_y

In [8]:
def split(df_x,df_y,num):
    # train-valid split, use last 28 days as validation
    df_xtrain = df_x.iloc[num:-28,:]
    df_xvalid = df_x.iloc[-28:,:]
    df_ytrain = df_y.iloc[num:-28]
    df_yvalid = df_y.iloc[-28:]
    
    return df_xtrain,df_ytrain,df_xvalid,df_yvalid

In [9]:
def rmsse(y_train,y_test,y_pred):
    y_train1 = y_train.shift()
    n = len(y_train)
    h = len(y_test)
    dividor = sum((y_train[1:]-y_train1[1:])**2)/(n-1)
    divident = sum((y_test-y_pred)**2)/h
    
    return divident/dividor

In [17]:
def gb_select(df_xtrain,df_ytrain,df_xvalid,df_yvalid,m,a):
    model = gbr(random_state=1,max_features=m,n_estimators=150,learning_rate=a).fit(df_xtrain,df_ytrain)
    df_ypred = model.predict(df_xvalid)

    return rmsse(df_ytrain,df_yvalid,df_ypred)

In [11]:
weight = np.array([0.110888286,0.110246513,0.155628193,0.065608342,0.0775605,0.09520139,0.096534717,0.087117424,0.116621569,0.084593065])


In [12]:
error_table = pd.DataFrame(index=range(1,12),columns = [0.001,0.003,0.005,0.007,0.01,0.03,0.05,0.07,0.1,0.3])

In [24]:
for m in range(1,12):
    for a in [0.001,0.003,0.005,0.007,0.01,0.03,0.05,0.07,0.1,0.3]:
        b = []
        Loss = np.array([0])
        for i in range(len(dfs)):
            df_x,df_y = clean(dfs[i])
            df_xtrain,df_ytrain,df_xvalid,df_yvalid = split(df_x,df_y,-759)
            loss= gb_select(df_xtrain,df_ytrain,df_xvalid,df_yvalid,m,a)
            b.append(loss)
        Loss = np.array(b)
        error_table.loc[m,a] = sum(Loss*weight)

In [25]:
error_table

,0.001,0.003,0.005,0.007,0.010,0.030,0.050,0.070,0.100,0.300
1,1.63531,1.36277,1.14881,0.98292,0.797748,0.366524,0.330203,0.308994,0.358738,0.471368
2,1.5802,1.23404,0.979548,0.794606,0.619686,0.326458,0.324346,0.361624,0.356639,0.474688
3,1.53446,1.13407,0.863709,0.686528,0.514657,0.274103,0.283623,0.314702,0.36724,0.477127
4,1.50518,1.08121,0.802253,0.619238,0.458976,0.263615,0.281711,0.330633,0.297381,0.512497
5,1.48776,1.05136,0.779496,0.608482,0.456841,0.274459,0.266453,0.29819,0.313272,0.494746
6,1.48352,1.04072,0.780496,0.617476,0.457813,0.26163,0.265723,0.307995,0.348016,0.37588
7,1.48134,1.04281,0.77833,0.613169,0.458931,0.248213,0.273304,0.26849,0.282916,0.347218
8,1.47847,1.04602,0.790436,0.614902,0.460509,0.248815,0.2935,0.315264,0.389595,0.459114
9,1.4794,1.058,0.796521,0.622839,0.468472,0.245677,0.265735,0.306517,0.365231,0.404595
10,1.47954,1.0667,0.807042,0.637957,0.492014,0.248104,0.279808,0.339377,0.298477,0.485684


In [27]:
for a in [0.02,0.04]:
    print('a=',a)
    b = []
    Loss = np.array([0])
    for i in range(len(dfs)):
        df_x,df_y = clean(dfs[i])
        df_xtrain,df_ytrain,df_xvalid,df_yvalid = split(df_x,df_y,-759)
        loss= gb_select(df_xtrain,df_ytrain,df_xvalid,df_yvalid,9,a)
        b.append(loss)
    Loss = np.array(b)
    print(sum(Loss*weight))

a= 0.02
0.2561138894131205
a= 0.04
0.24908494634554074


In [26]:
valid_train = pd.DataFrame(columns = ['wday', 'month', 'year', 'mday', 'day', 'week', 'national', 'snap_CA','snap_TX', 'snap_WI', 'demand_28'])

date = []
for i in range(1,57):
    date.append(pd.to_datetime('2016-04-24')+timedelta(days=i))
wday = []
for x in date:
    num = int(x.strftime("%w"))
    if num == 5:
        wday.append(7)
    else:
        wday.append((num+2)%7)
valid_train['wday'] = wday

month = []
for x in date:
    month.append(int(x.strftime("%m")))
valid_train['month'] = month

valid_train['year'] = [2016]*56

mday = []
for x in date:
    mday.append(int(x.strftime("%d")))
valid_train['mday'] = mday

valid_train['day'] = range(1914,1914+56)

week = [13]*5+[14]*7+[15]*7+[16]*7+[17]*7+[18]*7+[19]*7+[20]*7+[21]*2
valid_train['week'] = week

# only 5/30/2016 is a federal holiday
national = [0]*36+[1]+[0]*19
valid_train['national'] = national

mask1 = (valid_train.mday<=10)
valid_train['snap_CA']=np.where(mask1,1,0)

mask2 = (valid_train.mday==1)|(valid_train.mday==3)|(valid_train.mday==5)|(valid_train.mday==6)|(valid_train.mday==7)|(valid_train.mday==9)|(valid_train.mday==11)|(valid_train.mday==12)|(valid_train.mday==13)|(valid_train.mday==15)
valid_train['snap_TX']=np.where(mask2,1,0)

mask3 = (valid_train.mday==2)|(valid_train.mday==3)|(valid_train.mday==5)|(valid_train.mday==6)|(valid_train.mday==8)|(valid_train.mday==9)|(valid_train.mday==11)|(valid_train.mday==12)|(valid_train.mday==14)|(valid_train.mday==15)
valid_train['snap_WI']=np.where(mask3,1,0)

valid_train['demand_28'] = [0]*56

In [28]:
def gb_forecast(df_x,df_y,valid_train):
    model = gbr(random_state=1,max_features=9,n_estimators=150,learning_rate=0.03).fit(df_x,df_y)
    valid_train.demand_28[:28] = df_y[-28:]
    df_ypred1 = model.predict(valid_train.iloc[:28,:])
    valid_train.demand_28[28:] = df_ypred1
    df_ypred = model.predict(valid_train)

    return df_ypred

In [29]:
pred = pd.DataFrame(columns=['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3'])

In [30]:
dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
stores = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
for i in range(len(dfs)):
    df_x,df_y = clean(dfs[i])
    df_ypred = gb_forecast(df_x,df_y,valid_train)
    pred[f'{stores[i]}'] = df_ypred

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

In [31]:
pred

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,4014.384492,3711.822590,5781.222438,2580.959254,2964.424790,3661.177349,3559.145377,3090.542558,4299.913193,2993.178987
1,3757.392577,3684.207375,5497.756185,2376.110117,2677.983325,3149.394367,3124.812546,2901.397452,4147.699343,3158.972542
2,3506.296115,3647.485522,5387.167729,2343.148030,2724.547257,3339.687108,3028.432361,2696.368231,3893.893039,2946.233275
3,3770.648874,3647.485522,5449.814972,2358.504009,2701.376457,3339.687108,3191.833545,2830.391706,3979.131433,2881.068504
4,4309.677044,4638.355953,5707.029928,2385.415507,2973.252937,3756.658887,3528.628558,3840.586514,4719.658168,3593.822424
5,5513.223526,5988.621587,7164.325445,2866.065306,3623.329128,4475.008178,4067.798489,4913.644111,5203.936313,4374.097053
6,5858.653014,6050.252495,7797.295972,2957.502280,4067.680275,4858.883577,4519.038141,4677.007867,5612.599078,4693.526694
7,4826.092269,3944.755646,6738.283349,2742.549048,3052.763224,3738.601647,3760.984189,3320.860777,5858.256145,3840.328715
8,4142.156121,3662.714482,6348.084783,2523.416379,3134.261766,3759.520358,3836.471984,3326.526592,5989.229840,3950.493943
9,4018.983739,3673.152896,5880.214337,2426.783307,2954.955062,3390.185113,3468.979895,3099.101232,5030.370317,3499.236517


In [32]:
pred = pred.transpose()
pred.to_csv('store_pred_gb.csv')

In [33]:
df0 = pd.read_csv('data_CA_1.csv')
df1 = pd.read_csv('data_CA_2.csv')
df2 = pd.read_csv('data_CA_3.csv')
df3 = pd.read_csv('data_CA_4.csv')
df4 = pd.read_csv('data_TX_1.csv')
df5 = pd.read_csv('data_TX_2.csv')
df6 = pd.read_csv('data_TX_3.csv')
df7 = pd.read_csv('data_WI_1.csv')
df8 = pd.read_csv('data_WI_2.csv')
df9 = pd.read_csv('data_WI_3.csv')

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [34]:
def get_ratio(df):
    df['day'] = df.d.apply(lambda x: x[2:])
    df.day = df.day.astype(int)
    last = df[df.day>1913-28]
    sum_demand = sum(last.demand)
    sep_demand = last.groupby(['id']).sum().demand
    
    sep_demand['ratio'] = sep_demand/sum_demand
        
    return sep_demand['ratio']

In [35]:
def forecast(df,store_name):
    result = pd.DataFrame(columns=range(1,57))
    ratio = get_ratio(df)
    for x in range(1,57):
        result[x]=ratio*pred.loc[store_name,:][x-1]
    return result

In [36]:
result = pd.DataFrame(columns=range(1,57))

dfs = [df0,df1,df2,df3,df4,df5,df6,df7,df8,df9]
stores = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']

for i in range(len(dfs)):
    sub_data = forecast(dfs[i],stores[i])
    result = pd.concat([result,sub_data])

In [37]:
result

,1,2,3,4,5,6,7,8,9,10,...,47,48,49,50,51,52,53,54,55,56
FOODS_1_001_CA_1,1.035322,0.969043,0.904285,0.972462,1.111479,1.421878,1.510965,1.244664,1.068275,1.036509,...,1.129472,1.463720,1.437273,1.141770,0.979505,0.979505,0.979505,1.119268,1.421513,1.430216
FOODS_1_002_CA_1,0.376481,0.352379,0.328831,0.353623,0.404174,0.517046,0.549442,0.452605,0.388464,0.376912,...,0.410717,0.532262,0.522645,0.415189,0.356184,0.356184,0.356184,0.407007,0.516914,0.520078
FOODS_1_003_CA_1,0.690215,0.646029,0.602857,0.648308,0.740986,0.947919,1.007310,0.829776,0.712183,0.691006,...,0.752981,0.975813,0.958182,0.761180,0.653003,0.653003,0.653003,0.746179,0.947675,0.953477
FOODS_1_004_CA_1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_1_005_CA_1,1.066696,0.998408,0.931687,1.001931,1.145161,1.464965,1.556752,1.282382,1.100647,1.067918,...,1.163699,1.508075,1.480826,1.176369,1.009187,1.009187,1.009187,1.153185,1.464589,1.473556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3,0.160621,0.169518,0.158102,0.154605,0.192853,0.234725,0.251866,0.206081,0.211993,0.187778,...,0.204173,0.259844,0.253141,0.181417,0.189684,0.193456,0.171549,0.198544,0.241943,0.226404
HOUSEHOLD_2_513_WI_3,0.240932,0.254277,0.237153,0.231908,0.289280,0.352087,0.377799,0.309122,0.317990,0.281666,...,0.306259,0.389766,0.379712,0.272125,0.284526,0.290184,0.257324,0.297816,0.362915,0.339606
HOUSEHOLD_2_514_WI_3,0.133851,0.141265,0.131752,0.128838,0.160711,0.195604,0.209889,0.171735,0.176661,0.156481,...,0.170144,0.216536,0.210951,0.151181,0.158070,0.161213,0.142958,0.165453,0.201619,0.188670
HOUSEHOLD_2_515_WI_3,0.133851,0.141265,0.131752,0.128838,0.160711,0.195604,0.209889,0.171735,0.176661,0.156481,...,0.170144,0.216536,0.210951,0.151181,0.158070,0.161213,0.142958,0.165453,0.201619,0.188670


In [38]:
result.to_csv('result_gb.csv')

In [39]:
validation = result.loc[:,:28]
evaluation = result.loc[:,29:]

In [40]:
validation.reset_index(inplace=True)
for i in range(30490):
    validation['index'][i] = validation['index'][i]+'_validation'
validation.rename(columns = {'index':'id'},inplace = True)
validation.set_index('id',inplace=True)

evaluation.reset_index(inplace=True)
for i in range(30490):
    evaluation['index'][i] = evaluation['index'][i]+'_evaluation'
evaluation.rename(columns = {'index':'id'},inplace=True)
evaluation.set_index('id',inplace=True)

/Users/liuyikun/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/liuyikun/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata

In [41]:
columns = []
for i in range(1,29):
    columns.append(f'F{i}')
validation.columns = columns
evaluation.columns = columns
result=pd.concat([validation,evaluation])

In [42]:
result

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,1.035322,0.969043,0.904285,0.972462,1.111479,1.421878,1.510965,1.244664,1.068275,1.036509,...,1.058900,1.420761,1.447341,1.128030,0.978753,0.974968,0.974968,1.119869,1.420761,1.447341
FOODS_1_002_CA_1_validation,0.376481,0.352379,0.328831,0.353623,0.404174,0.517046,0.549442,0.452605,0.388464,0.376912,...,0.385055,0.516640,0.526306,0.410193,0.355910,0.354534,0.354534,0.407225,0.516640,0.526306
FOODS_1_003_CA_1_validation,0.690215,0.646029,0.602857,0.648308,0.740986,0.947919,1.007310,0.829776,0.712183,0.691006,...,0.705933,0.947174,0.964894,0.752020,0.652502,0.649979,0.649979,0.746580,0.947174,0.964894
FOODS_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_1_005_CA_1_validation,1.066696,0.998408,0.931687,1.001931,1.145161,1.464965,1.556752,1.282382,1.100647,1.067918,...,1.090988,1.463814,1.491199,1.162213,1.008412,1.004513,1.004513,1.153805,1.463814,1.491199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3_evaluation,0.163871,0.164319,0.158102,0.158102,0.184733,0.227830,0.221107,0.167535,0.157547,0.181823,...,0.204173,0.259844,0.253141,0.181417,0.189684,0.193456,0.171549,0.198544,0.241943,0.226404
HOUSEHOLD_2_513_WI_3_evaluation,0.245806,0.246479,0.237153,0.237153,0.277100,0.341746,0.331661,0.251302,0.236321,0.272735,...,0.306259,0.389766,0.379712,0.272125,0.284526,0.290184,0.257324,0.297816,0.362915,0.339606
HOUSEHOLD_2_514_WI_3_evaluation,0.136559,0.136933,0.131752,0.131752,0.153944,0.189859,0.184256,0.139612,0.131290,0.151519,...,0.170144,0.216536,0.210951,0.151181,0.158070,0.161213,0.142958,0.165453,0.201619,0.188670


In [43]:
result.to_csv('final_result_gb.csv')

In [46]:
result_rf = pd.read_csv('final_result.csv',index_col='id')

In [47]:
result_rf

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001_CA_1_validation,1.065713,0.960542,0.941414,0.953700,1.121917,1.426460,1.551685,1.277977,1.035904,1.019238,...,1.135594,1.449741,1.467887,1.137469,0.953009,0.947763,0.951119,1.150975,1.441176,1.453148
FOODS_1_002_CA_1_validation,0.387532,0.349288,0.342332,0.346800,0.407970,0.518713,0.564249,0.464719,0.376692,0.370632,...,0.412943,0.527178,0.533777,0.413625,0.346549,0.344641,0.345861,0.418537,0.524064,0.528417
FOODS_1_003_CA_1_validation,0.710475,0.640362,0.627609,0.635800,0.747945,0.950973,1.034456,0.851985,0.690603,0.679492,...,0.757063,0.966494,0.978591,0.758313,0.635339,0.631842,0.634079,0.767317,0.960784,0.968765
FOODS_1_004_CA_1_validation,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
FOODS_1_005_CA_1_validation,1.098007,0.989650,0.969941,0.982600,1.155915,1.469686,1.598705,1.316704,1.067295,1.050124,...,1.170006,1.493672,1.512368,1.171938,0.981888,0.976483,0.979940,1.185853,1.484848,1.497182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512_WI_3_evaluation,0.167236,0.168384,0.163147,0.163437,0.195461,0.220885,0.220797,0.164807,0.159563,0.182973,...,0.199918,0.276723,0.263960,0.192151,0.201507,0.218816,0.180021,0.202942,0.245962,0.235298
HOUSEHOLD_2_513_WI_3_evaluation,0.250855,0.252576,0.244720,0.245155,0.293192,0.331328,0.331196,0.247211,0.239344,0.274459,...,0.299877,0.415084,0.395940,0.288227,0.302261,0.328224,0.270032,0.304413,0.368943,0.352946
HOUSEHOLD_2_514_WI_3_evaluation,0.139364,0.140320,0.135956,0.136197,0.162884,0.184071,0.183998,0.137339,0.132969,0.152477,...,0.166598,0.230602,0.219967,0.160126,0.167923,0.182346,0.150018,0.169119,0.204968,0.196081
